# Importing libaries

In [123]:
#data processing and importing
import pandas as pd
import numpy as np
import csv

#label encoder
from sklearn import preprocessing

#text transformation via spacy 
import torch
import spacy
import spacy_transformers
import de_dep_news_trf
from keras.layers import TextVectorization
from keras.preprocessing import sequence

#lstm model 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

***

# Importing and preparing the data

*Importing raw data*

In [90]:
#import speeches for 2021 as sample
speeches_df = pd.read_csv('data/speeches_new.csv')
speeches_df.columns

Index(['id', 'session', 'electoralTerm', 'firstName', 'lastName',
       'politicianId', 'speechContent', 'factionId', 'documentUrl',
       'positionShort', 'positionLong', 'date'],
      dtype='object')

In [91]:
#import faction id 
faction_df = pd.read_csv('data/factions.csv')
faction_df = faction_df.rename(columns={'id':'factionId'})

In [92]:
#merging the two dataframes
data = pd.merge(left = speeches_df, right = faction_df, on = 'factionId', how = 'inner')

In [93]:
#dropping all columns that are not needed
data = data[['speechContent', 'abbreviation']]
data.rename(columns={'abbreviation':'partie'}, inplace=True)

In [94]:
#expecting the value count of all columns 
data.partie.value_counts()

not found     4195
CDU/CSU       1086
SPD            704
FDP            558
AfD            545
Grüne          544
DIE LINKE.     486
Name: partie, dtype: int64

In [95]:
#removing not found
data = data[data.partie != 'not found']

*DataFrame of imported data*

In [101]:
#creating small samples of speeches, for each political party only 50 
max_number_speeches = 50

cdu_csu = data[data['partie'] == 'CDU/CSU'][:max_number_speeches]
spd = data[data['partie'] == 'SPD'][:max_number_speeches]
fdp = data[data['partie'] == 'FDP'][:max_number_speeches]
afd = data[data['partie'] == 'AfD'][:max_number_speeches]
gruene = data[data['partie'] == 'Grüne'][:max_number_speeches]
linke = data[data['partie'] == 'DIE LINKE.'][:max_number_speeches]

data = pd.concat([cdu_csu, spd, fdp, afd, gruene, linke], ignore_index=True)

In [97]:
#defining X_df and y_df
X_df = data['speechContent']
y_df = data['partie'] 

***

# Preprocessing the speeches (X)

Remarks: Since I want to include grammatical information in my model’s input vectors, I don’t want to remove stopwords or lemmatize words. Also, as capitilazation is very crucial in the German language, I did not lower the single words. I only removed special characters via RegEx, did vectorizing via word embedding and padding. 

*RegEx*

In [98]:
#examining X_df
X_df.head()

0    \n\nHerr Präsident! Meine Damen! Meine Herren!...
1    \n\nHerr Präsident! Liebe Kolleginnen und Koll...
2    \n\nHerr Präsident! Liebe Kolleginnen und Koll...
3    \n\nSehr geehrter Herr Präsident! Sehr geehrte...
4    \n\nFrau Präsidentin! Liebe Kolleginnen und Ko...
Name: speechContent, dtype: object

In [99]:
#cleaning the speeches using regex
X_df = X_df.str.replace('\n',' ', regex = True) #removes all newlines 
X_df = X_df.str.replace('\xa0',' ', regex = True) #removes all \xa0
X_df = X_df.str.replace(',',' ', regex = True) #removes all commas
X_df = X_df.str.replace('?',' ', regex = True) #removes all ?
X_df = X_df.str.replace('!',' ', regex = True) #removes all !
X_df = X_df.str.replace('.',' ', regex = True) #removes all .
X_df = X_df.str.replace('–',' ', regex = True) #removes all -
X_df = X_df.str.replace(':',' ', regex = True) #removes all :
X_df = X_df.str.replace(';',' ', regex = True) #removes all ;
X_df = X_df.str.replace('„',' ', regex = True) #removes all „
X_df = X_df.str.replace('“',' ', regex = True) #removes all “
X_df = X_df.str.replace('/',' ', regex = True) #removes all /
X_df = X_df.str.replace(r"\(.*\)"," ") #removes brackets and everything that is inside of it
X_df = X_df.str.replace(r"\[.*\]"," ") #removes squared brackets and everything that is inside of it

/var/folders/n_/f0kktvdx00s93tvvh3xlcch40000gn/T/ipykernel_53262/1883932841.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  X_df = X_df.str.replace(r"\(.*\)"," ") #removes brackets and everything that is inside of it
/var/folders/n_/f0kktvdx00s93tvvh3xlcch40000gn/T/ipykernel_53262/1883932841.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  X_df = X_df.str.replace(r"\[.*\]"," ") #removes squared brackets and everything that is inside of it


In [100]:
#transform the dataframe series into lists & checks
X = X_df.to_list()  #transform the dataframe series into lists
len(X)              #checking if the length of the list is still 300 
X[51]               #expecting single speeches for check-up

'  Herr Präsident  Liebe Kolleginnen und Kollegen  Ich bin dem Bundesgesundheitsminister dankbar  dass er eine Regierungserklärung zum Impfstart abgegeben hat  Das gibt uns die Gelegenheit zu einer umfassenden Aussprache  Ich halte das für notwendig   Die Situation in Deutschland und in Europa ist immer noch bedrohlich  Die Mutationen des Coronavirus  die in Großbritannien  in Irland  aber auch in Südafrika neu entdeckt wurden und dort zu einer extrem hohen Ansteckungsgefahr und höheren Infiziertenzahlen führen  haben dazu geführt  dass wir über einen Wettlauf mit der Zeit reden  einen Wettlauf mit der Zeit um Menschenleben  einen Wettlauf  bei dem es darum geht  wieder zum normalen Leben zurückzukommen   Die Einschränkungen  die wir haben  sind gewaltig  ökonomisch  sozial und insbesondere bildungspolitisch für Kinder aus Familien  die nicht über die Kompetenzen verfügen und vor allen Dingen nicht die digitalen Voraussetzungen haben  um schulische Veranstaltungen  wie sie derzeit nur 

***

*Vectorizing via Word Embedding*

Remarks: The TextVectorization (using keras) layer will tokenize, vectorize, and pad sequences representing those documents to be passed to the embedding layer (using Spacy).

In [184]:
#loading the needed libaries
nlp = spacy.load('de_core_news_md') #spacy
Vectorizer = TextVectorization()    #keras.TextVectorization 

In [198]:
#fit the vectorizer on the text and extract the corpus vocabulary
Vectorizer.adapt(X)
vocab = Vectorizer.get_vocabulary() 
vocab                             #inspecting the first tokens

['',
 '[UNK]',
 'die',
 'und',
 'der',
 'das',
 'in',
 'sie',
 'ist',
 'wir',
 'es',
 'nicht',
 'den',
 'dass',
 'zu',
 'auch',
 'haben',
 'ich',
 'von',
 'mit',
 'ein',
 'sehr',
 'für',
 'eine',
 'auf',
 'im',
 'sind',
 'herr',
 'sich',
 'hat',
 'wie',
 'dem',
 'werden',
 'aber',
 'kollegen',
 'frau',
 'des',
 'präsident',
 'meine',
 'kolleginnen',
 'herren',
 'damen',
 'um',
 'an',
 'bei',
 'als',
 'was',
 'hier',
 'man',
 'ja',
 'noch',
 'uns',
 'diese',
 'liebe',
 'zum',
 'so',
 'aus',
 'vor',
 'wenn',
 'wird',
 'schon',
 'oder',
 'jetzt',
 'über',
 'einen',
 'nur',
 'präsidentin',
 'können',
 'dieser',
 'da',
 'dann',
 'muss',
 'menschen',
 'mehr',
 'kann',
 'ganz',
 'frage',
 'einer',
 'dank',
 'mal',
 'zur',
 'ihnen',
 'geht',
 'geehrte',
 'einem',
 'wollen',
 'ihre',
 'geehrter',
 'müssen',
 'deutschland',
 'heute',
 'durch',
 'denn',
 'sagen',
 'immer',
 'am',
 'sondern',
 'keine',
 'antrag',
 'viel',
 'ministerin',
 'also',
 'alle',
 'thema',
 'vielen',
 'war',
 'nach',
 'mac

In [186]:
#how many tokens (single words) are there?
token_count= len(vocab)
print(f'The Vectorizer produced {token_count} tokens.')

The Vectorizer produced 5872 tokens.


In [187]:
#generate the embedding matrix
embedding_dim = len(nlp('test').vector)
embedding_dim

300

In [193]:
#creating an empty embedding matrix in the size of num_tokens & embedding_dim
embedding_matrix = np.zeros((token_count, embedding_dim))
embedding_matrix.shape

(5872, 300)

In [202]:
#mapping the empty embedding matrix to the nlp word vector
for i, word in enumerate(vocab):
    print(i, word)
    embedding_matrix[i] = nlp(str(word)).vector
    

0 
1 [UNK]
2 die
3 und
4 der
5 das
6 in
7 sie
8 ist
9 wir
10 es
11 nicht
12 den
13 dass
14 zu
15 auch
16 haben
17 ich
18 von
19 mit
20 ein
21 sehr
22 für
23 eine
24 auf
25 im
26 sind
27 herr
28 sich
29 hat
30 wie
31 dem
32 werden
33 aber
34 kollegen
35 frau
36 des
37 präsident
38 meine
39 kolleginnen
40 herren
41 damen
42 um
43 an
44 bei
45 als
46 was
47 hier
48 man
49 ja
50 noch
51 uns
52 diese
53 liebe
54 zum
55 so
56 aus
57 vor
58 wenn
59 wird
60 schon
61 oder
62 jetzt
63 über
64 einen
65 nur
66 präsidentin
67 können
68 dieser
69 da
70 dann
71 muss
72 menschen
73 mehr
74 kann
75 ganz
76 frage
77 einer
78 dank
79 mal
80 zur
81 ihnen
82 geht
83 geehrte
84 einem
85 wollen
86 ihre
87 geehrter
88 müssen
89 deutschland
90 heute
91 durch
92 denn
93 sagen
94 immer
95 am
96 sondern
97 keine
98 antrag
99 viel
100 ministerin
101 also
102 alle
103 thema
104 vielen
105 war
106 nach
107 machen
108 jahr
109 weil
110 dazu
111 gibt
112 diesem
113 habe
114 gerade
115 wieder
116 unter
117 dieses
118 d

***

# Preprocessing the labels (y)

In [191]:
#transforming the parties into labels /// zahlen beispiel 
le = preprocessing.LabelEncoder()
y = y_df.to_list()
le.fit(y)
y = le.transform(y)

In [192]:
#inspecting labels aka y_train
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

***

# Building the LSTM model

In [203]:
model = Sequential()
#model.add(Embedding(input_dim=token_count, output_dim=300, trainable=False))     #trainable = False, for this layer not trained - also good against overfitting
model.add(LSTM(5))                         #first start small, than max up the number
model.add(Dense(6, activation='softmax'))   #softmax activation, last layer should have 6 neurons as there are 6 parties
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         1761600   
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 1,855,430
Trainable params: 93,830
Non-trainable params: 1,761,600
_________________________________________________________________
None


***

# Fitting the LSTM model

In [ ]:
#fit the model
#model.fit(X, y, batch_size = 50, epochs = 50, validation_split = 0.2)